# 05 - Delta Hedging Error Lab

Study discrete hedging error versus rebalance frequency and transaction costs under GBM.

In [ ]:
from pathlib import Path
import sys

ROOT = Path.cwd()
if not (ROOT / 'src').exists():
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

import pandas as pd

from src.config import config_dict
from src.hedging import hedging_experiment_grid, simulate_delta_hedge_gbm
from src.plotting import plot_hedging_error_hist, plot_hedging_tradeoff

cfg = config_dict(fast_mode=True)


In [ ]:
grid = hedging_experiment_grid(
    S0=cfg['S0'], K=cfg['K'], r=cfg['R'], q=cfg['Q'], sigma=cfg['SIGMA'], T=cfg['T'],
    n_paths=8_000, n_steps=252,
    rebalance_list=cfg['HEDGE_REBALANCE_STEPS'],
    tx_cost_list=[0.0, 0.001, 0.003],
    option_type='call',
    seed=cfg['SEED'],
)
grid.to_csv(ROOT / 'results' / 'tables' / 'hedging_error_summary.csv', index=False)
plot_hedging_tradeoff(grid, ROOT / 'results' / 'figures' / 'hedging_tradeoff.png')
grid.head()


In [ ]:
base = simulate_delta_hedge_gbm(
    S0=cfg['S0'], K=cfg['K'], r=cfg['R'], q=cfg['Q'], sigma=cfg['SIGMA'], T=cfg['T'],
    n_paths=8_000, n_steps=252, rebalance_every_k_steps=5,
    option_type='call', tx_cost_per_dollar=0.0, seed=cfg['SEED']
)
plot_hedging_error_hist(base['errors'], ROOT / 'results' / 'figures' / 'hedging_error_hist.png')
print('Mean error:', base['mean_error'])
print('Std error:', base['std_error'])
